# 02. 플라스크(Flask)로 웹 API 만들기
- http://flask.pocoo.org/ - Quickstart에 있는 코드를 `hello.py`에 저장

```python
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

if __name__ == '__main__':
    app.run()
```

- `python hello.py`으로 실행
- http://127.0.0.1:5000/ 에 접속하여 Hello World! 텍스트가 출력되는 것을 확인하면 정상 실행된 것

## 데이터베이스 연결하기
- 플라스크에는 MySQL 데이터베이스에 연결할 수 있게 해주는 기능이 없기 때문에 Peewee라는 ORM 라이브러리를 사용
    - ORM 라이브러리 : '관계형 데이터베이스의 데이터'를 '객체 지향 언어의 클래스와 같은 데이터'로 매핑하기 위한 라이브러리
        - SQL 구문을 사용하지 않아도 데이터베이스에서 데이터를 추출하거나 추가할 수 있음

### 데이터베이스 조작 전용 파일 만들기
- `db.py`

```python
import peewee
from playhouse.pool import PooledMySQLDatabase

# sakila 데이터베이스에 접근하기 위한 정보
db = PooledMySQLDatabase(
    'sakila',
    max_connections=8,
    stale_timeout=10,
    user='root',
    password='3756')


class BaseModel(peewee.Model):
    """기본 부모 모델"""
    class Meta:
        database = db
        

class Language(BaseModel):
    """language 테이블 전용 모델"""
    
    language_id = peewee.SmallIntegerField(primary_key=True)
    name = peewee.CharField(max_length=20)
    last_update = peewee.TimestampField()
    
    class Meta:
        db_table = 'language'
        
        
class Film(BaseModel):
    """film 테이블 전용 모델"""
    
    film_id = peewee.SmallIntegerField(primary_key=True)
    title = peewee.CharField(index=True)
    description = peewee.TextField(null=True)
    release_year = peewee.DateField(formats='%Y')
    
    # 외부키
    language = peewee.ForeignKeyField(Language)
    length = peewee.SmallIntegerField()
    last_update = peewee.TimestampField()
    
    def to_dict(self):
        return {
            'film_id':self.film_id,
            'title':self.title,
            'description':self.description,
            'release_year':self.release_year,
            'language':self.language.name,
            'length':self.length,
            'last_update':self.last_update.isoformat(),
        }
    
    class Meta:
        db_table = 'film'
```

### iPython을 사용해서 아이템 추출하기
- `ipython`으로 iPython 실행

```ipython
In [1]: from db import Film

In [2]: film = Film.get(Film.film_id == 10)

In [3]: film.film_id
Out[3]: 10

In [4]: film.title
Out[4]: 'ALADDIN CALENDAR'

In [5]: film.to_dict()
Out[5]:
{'film_id': 10,
 'title': 'ALADDIN CALENDAR',
 'description': 'A Action-Packed Tale of a Man And a Lumberjack who must Reach a Feminist in Ancient China',
 'release_year': 2006,
 'language': 'English',
 'length': 63,
 'last_update': '2006-02-15T05:03:42'}

In [6]: films = Film.select().limit(3)

In [7]: [film.to_dict() for film in films]
Out[7]:
[{'film_id': 1,
  'title': 'ACADEMY DINOSAUR',
  'description': 'A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies',
  'release_year': 2006,
  'language': 'English',
  'length': 86,
  'last_update': '2006-02-15T05:03:42'},
 {'film_id': 2,
  'title': 'ACE GOLDFINGER',
  'description': 'A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China',
  'release_year': 2006,
  'language': 'English',
  'length': 48,
  'last_update': '2006-02-15T05:03:42'},
 {'film_id': 3,
  'title': 'ADAPTATION HOLES',
  'description': 'A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory',
  'release_year': 2006,
  'language': 'English',
  'length': 50,
  'last_update': '2006-02-15T05:03:42'}]
```

## 플라스크 플러그인: Flask-RESTful 사용하기
- https://github.com/humiaozuzu/awesome-flask 에 다양한 플러그인이 있음
- Flask-RESTful은 RESTful 설계 사상을 기반으로 웹 API를 쉽게 만들 수 있게 해주는 플러그인
- `pip install flask-restful`

### 플라스크 애플리케이션 만들기
- `flask_film_api.py`

```python
from flask import Flask, abort
from flask_restful import Api, Resource, reqparse

# db.py를 기반으로 Film 테이블 모델 전용 클래스 읽어 들이기
from db import Film
app = Flask(__name__)

# RESTful API 전용 인터페이스 만들기
api = Api(app)

# 단일 아이템을 출력하는 클래스
class FilmItem(Resource):
    """Film 아이템"""
    def get(self, film_id):
        """GET 실행 때의 액션"""
        try:
            # film 테이블에서 film_id를 검색해서 데이터베이스에서 추출하기
            film = Film.get(Film.film_id==film_id)
        except Film.DoesNotExist:
            # 없는 경우 404 응답
            abort(404, description="Film not found.")
        
        # 딕셔너리 자료형으로 반환하면 자동으로 JSON으로 변환됨
        return film.to_dict()
    
# 여러 개의 아이템을 출력하는 클래스
class FilmList(Resource):
    """Film 목록"""
    # 한 페이지 당 아이템 수
    ITEMS_PER_PAGE = 5
    
    def __init__(self, *args, **kwargs):
        """GET 요청 때의 요청 매개 변수 파싱하기"""
        self.parser = reqparse.RequestParser()
        
        # 이렇게 하면 ?page=2 등으로 추가적인 정보를 전달 받을 수 있음
        self.parser.add_argument('page', type=int, default=1)
        super().__init__(*args, **kwargs)
        
    def get(self):
        """GET 실행 때의 액션"""
        # GET 요청 때의 요청 매개 변수 추출하기
        args = self.parser.parse_args()
        
        # film 테이블에서 page 요청 매개 변수에 따라 페이징해서 아이템 5개씩 추출
        films = Film.select().order_by(Film.film_id).paginate(args['page'], self.ITEMS_PER_PAGE) # 페이징 처리
        
        # 딕셔너리 자료형으로 반환하면 자동으로 JSON으로 변환됨
        return [film.to_dict() for film in films]
    
# 어떤 URL 형식으로 접근할 때, 어떤 것을 실행할지 지정하기
api.add_resource(FilmItem, '/film/<int:film_id>')
api.add_resource(FilmList, '/films')


if __name__ == '__main__':
    # 웹 서버 실행하기
    app.run(debug=True)
```

- `python flask_film_api.py` 실행 후 http://127.0.0.1:5000/film/10 에 접속하면 아래와 같은 응답을 받으며

```
{
    "film_id": 10,
    "title": "ALADDIN CALENDAR",
    "description": "A Action-Packed Tale of a Man And a Lumberjack who must Reach a Feminist in Ancient China",
    "release_year": 2006,
    "language": "English",
    "length": 63,
    "last_update": "2006-02-15T05:03:42"
}
```
- http://127.0.0.1:5000/films?page=2 에 접속하면 아래와 같은 응답을 받음

```
[
    {
        "film_id": 6,
        "title": "AGENT TRUMAN",
        "description": "A Intrepid Panorama of a Robot And a Boy who must Escape a Sumo Wrestler in Ancient China",
        "release_year": 2006,
        "language": "English",
        "length": 169,
        "last_update": "2006-02-15T05:03:42"
    },
    {
        "film_id": 7,
        "title": "AIRPLANE SIERRA",
        "description": "A Touching Saga of a Hunter And a Butler who must Discover a Butler in A Jet Boat",
        "release_year": 2006,
        "language": "English",
        "length": 62,
        "last_update": "2006-02-15T05:03:42"
    },
    {
        "film_id": 8,
        "title": "AIRPORT POLLOCK",
        "description": "A Epic Tale of a Moose And a Girl who must Confront a Monkey in Ancient India",
        "release_year": 2006,
        "language": "English",
        "length": 54,
        "last_update": "2006-02-15T05:03:42"
    },
    {
        "film_id": 9,
        "title": "ALABAMA DEVIL",
        "description": "A Thoughtful Panorama of a Database Administrator And a Mad Scientist who must Outgun a Mad Scientist in A Jet Boat",
        "release_year": 2006,
        "language": "English",
        "length": 114,
        "last_update": "2006-02-15T05:03:42"
    },
    {
        "film_id": 10,
        "title": "ALADDIN CALENDAR",
        "description": "A Action-Packed Tale of a Man And a Lumberjack who must Reach a Feminist in Ancient China",
        "release_year": 2006,
        "language": "English",
        "length": 63,
        "last_update": "2006-02-15T05:03:42"
    }
]
```